In [ ]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
# PyTorch related imports would go here
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
#from us import states
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re
import networkx as nx

In [ ]:
SETTINGS_NO = "Case8"

In [ ]:
# from google.colab import files

# # This will prompt you to upload a file from your local system
# uploaded = files.upload()
# filename = next(iter(uploaded))  # Get the first (and only) filename

# DATASET_NAME = "SPECIATE_V01282025_73_species.csv"
DATASET_NAME = "FinalDataset_transposed_02282025.csv"

df = pd.read_csv('PM25-Speciated/datasetasoffeb282025/'+DATASET_NAME, encoding='ISO-8859-1')

In [ ]:
# #DATASET_NAME = "FinalDataset_transposed_02282025.csv"
# DATASET_NAME =filename
# df = pd.read_csv(DATASET_NAME, encoding='ISO-8859-1')

In [ ]:
def clean_text(text):
    # Remove numbers and special characters
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [ ]:
unique_scc1_values = df['SCC2'].unique()
pivot_df = df
pivot_df['SCC1'] = pd.factorize(pivot_df['SCC1'])[0]
pivot_df['SCC3'] = pd.factorize(pivot_df['SCC3'])[0]

In [ ]:
# Step 2: Ensure SCC2 is treated as a string column
pivot_df['SCC2'] = pivot_df['SCC2'].astype(str)

# Split SCC2 based on ;
split_SCC2 = pivot_df['SCC2'].str.split(';', expand=True)

# Get unique sentence fragments
unique_fragments = pd.unique(split_SCC2.values.ravel('K'))

print(unique_fragments)

# Remove None values (if any)
unique_fragments = [fragment for fragment in unique_fragments if pd.notna(fragment) and fragment != 'nan']

# Create one-hot encoding for each fragment
one_hot_encoded = pd.DataFrame(0, index=pivot_df.index, columns=unique_fragments)

print(one_hot_encoded)

for col in split_SCC2.columns:
    for idx, fragment in split_SCC2[col].items():
        if pd.notna(fragment) and fragment != 'nan':
            one_hot_encoded.at[idx, fragment] = 1

# Step 3: Convert each row of one-hot encoding into a tuple
one_hot_tuples = [tuple(row) for row in one_hot_encoded.values]

# Step 4: Assign a unique sequential number to each unique combination
unique_combinations = pd.factorize(one_hot_tuples)[0]

# Add the sequential numbers to the DataFrame
pivot_df['SCC2'] = unique_combinations

# Display the transformed DataFrame
print(pivot_df)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer

print(pivot_df.columns[3:])

# imputer = IterativeImputer()
#imputer = SimpleImputer(strategy='median')  #  option 1
imputer = KNNImputer() # option 2
df_imputed = pd.DataFrame(imputer.fit_transform(pivot_df.iloc[:,3:]), columns=df.columns[3:])
pivot_df[pivot_df.columns[3:]]=df_imputed

# # Opton 3 for minimum
# min_values = pivot_df.iloc[:, 3:].min()
# imputers = {col: SimpleImputer(strategy='constant', fill_value=min_val) for col, min_val in min_values.items()}
# # Apply imputation column by column
# for col in pivot_df.columns[3:]:
#     pivot_df[[col]] = imputers[col].fit_transform(pivot_df[[col]])

pivot_df



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter

In [ ]:
def assign_cluster_name(cluster):
    names = pivot_df[pivot_df['speciated_cluster'] == cluster]['PROFILE_NAME']
    most_common_name = Counter(names).most_common(1)[0][0]
    return most_common_name

In [ ]:
com_map = {}
loop_info = []
#parameters
StartCol=3
k_SVD = 16
K_knn = 5

In [ ]:
#X = pivot_df.iloc[:, 3:]
## scaler = StandardScaler()
#scaler = MinMaxScaler()
#X = scaler.fit_transform(X)

In [ ]:
#pip install cdlib

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from scipy.sparse.linalg import svds

import cdlib
from cdlib import algorithms
import networkx as nx

In [ ]:
X = pivot_df.iloc[:, 6:]
X

In [ ]:
clustertype = "ClusterswoSCC"
folder = "PM25-Speciated/"+clustertype+"/"
# folder = "PM25-Speciated/ClusterswSCC/"

In [ ]:
# Assuming df and pivot_df are your dataframes loaded earlier

# Loop for different values of k_SVD, K_knn, and StartCol
k_SVD_values = [10, 15, 20,30]  # Example values for k_SVD
K_knn_values = [3, 4,5, 6,7,8,9]  # Example values for K_knn
# change this to 3 without SCC hotcoding data
StartCol_values = [6]#[3, 4,5, 6]  # Example start columns

AllGraphs=[]

for StartCol in StartCol_values:
    for k_SVD in k_SVD_values:
        for K_knn in K_knn_values:
            X = pivot_df.iloc[:, StartCol:]

            # Apply SVD to X
            U, S, VT = svds(X.values, k_SVD)
            X_svd = U @ np.diag(S) @ VT
            X = X_svd

            # Compute KNN
            knn = NearestNeighbors(n_neighbors=K_knn)  # You can change the number of neighbors
            knn.fit(X)
            distances, indices = knn.kneighbors(X)

            # Create a graph
            G = nx.Graph()

            # Add nodes with PROFILE_NAME as labels
            for i, profile_name in enumerate(df['PROFILE_NAME']):
                G.add_node(i, label=profile_name)

            # Add edges based on KNN
            for i, neighbors in enumerate(indices):
                for neighbor in neighbors:
                    if i != neighbor:  # Avoid self-loops
                        G.add_edge(i, neighbor, weight=1.0)
                        
            
            AllGraphs.append(G)
            

#             # Apply Louvain algorithm
#             coms = algorithms.louvain(G, weight='weight', resolution=1., randomize=False)
#             #print(coms)

#             for i in range(len(coms.communities)):
#                 community = coms.communities[i]
#                 for j in community:
#                     if j in com_map:
#                         com_map[j].append(i)
#                     else:
#                         com_map[j]=[i]
#             cluster_number = len(coms.communities)
#             #print(cluster_number)
#             print(f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}, No. of Clusters: {cluster_number} ")
#             print_string =f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}, No. of Clusters: {cluster_number} "
#             loop_info.append(print_string)

            print(f"Current StartCol: {StartCol}, Current k_SVD: {k_SVD}, Current K_knn: {K_knn}")
                        

In [ ]:
AllGraphs

In [ ]:
import networkx as nx
from networkx.utils import open_file
import itertools

In [ ]:
def combine_graphs(AllGraphs):
    """
    Combine multiple graphs using nx.compose_all() and sum edge weights.
    
    Args:
        AllGraphs: List of networkx graphs with the same nodes
        
    Returns:
        A new graph with combined edge weights
    """
    if not AllGraphs:
        return nx.Graph()
    
    # Use compose_all to combine graphs
    combined = nx.compose_all(AllGraphs)
    
    # For edges that exist in multiple graphs, sum their weights
    for u, v, data in combined.edges(data=True):
        data['weight'] = sum(G.get_edge_data(u, v, {}).get('weight', 1) 
                          for G in AllGraphs if G.has_edge(u, v))
    
    return combined

combined_graph = combine_graphs(AllGraphs)

combined_graph

In [ ]:
def print_edges_for_node_sorted(graph, node):
    if node not in graph:
        print(f"Node {node} not found in the graph")
        return
    
    edges = [(neighbor, data.get('weight', 1)) for neighbor, data in graph[node].items()]
    edges.sort(key=lambda x: x[1], reverse=True)  # Sort by weight descending
    
    print(f"Edges and weights for node {node} (sorted by weight):")
    for neighbor, weight in edges:
        print(f"({node}, {neighbor}): weight = {weight}")

# Example usage
print_edges_for_node_sorted(combined_graph, 0)

In [ ]:
for g in AllGraphs:
    print("*"*100)
    print_edges_for_node_sorted(g,0)

In [ ]:
@open_file(1, mode='wb')
def save_mm(graph, path):
    """
    Save graph in Matrix Market format with edge weights.
    
    Args:
        graph: NetworkX graph to save
        path: File path to save to
    """
    # Get the number of nodes and edges
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    
    # Create a mapping from nodes to integer indices
    node_index = {node: i+1 for i, node in enumerate(graph.nodes())}  # Matrix Market uses 1-based indexing
    
    # Write the header
    header = f"%%MatrixMarket matrix coordinate real symmetric\n%\n{num_nodes} {num_nodes} {num_edges}\n"
    path.write(header.encode('ascii'))
    
    # Write the edges
    for u, v, data in graph.edges(data=True):
        weight = data.get('weight', 1)
        path.write(f"{node_index[u]} {node_index[v]} {weight}\n".encode('ascii'))

# Example usage:
# Assuming AllGraphs is your list of graphs
save_mm(combined_graph, folder+"concensus-"+clustertype+".mtx")

In [ ]:
filename = folder+"concensus-"+clustertype+".gexf"
print("Writing gephi...."+filename)
nx.write_gexf(combined_graph, filename)
print("Done....")

## Apply Louvain algorithm

In [ ]:
fileno = 0

for w in [True, False]:
    
    print("weighted" if w else "unweighted")

    for resolution in [0.3, 0.5, 0.8, 1.0, 1.5, 2.0, 2.5, 3.0]: #default 1.0
        
        if w:
            coms = algorithms.louvain(combined_graph, weight='weight', resolution=resolution, randomize=False)
        else:
            coms = algorithms.louvain(combined_graph, resolution=resolution, randomize=False)
        #print(coms)
        
        # File path where you want to save the clusters
        output_file = folder+"cluster_no_"+str(fileno)+".txt"

        # Write clusters to the file
        with open(output_file, 'w') as f:
            for cluster in coms.communities:
                # Convert each node number to string and join with spaces
                line = ' '.join(map(str, cluster))
                f.write(line + '\n')

        print(f"Clusters written to {output_file}")

#         for i in range(len(coms.communities)):
#             community = coms.communities[i]
#             for j in community:
#                 if j in com_map:
#                     com_map[j].append(i)
#                 else:
#                     com_map[j]=[i]
        cluster_number = len(coms.communities)
        print(f"No. of Clusters: {cluster_number} ")
        fileno+=1

In [ ]:
# import csv
# # Writing com_map to CSV
# with open('com_map_KNNImputer_wSCC.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Node', 'Communities'])
#     for key, value in com_map.items():
#         writer.writerow([key, ', '.join(map(str, value))])


In [ ]:
# with open('loop_info_KNNImputer_wSCC.txt', mode='w') as file:
#     for line in loop_info:
#         file.write(line + '\n')

In [ ]:
# com_map

## Concensus clustering results

In [ ]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
# PyTorch related imports would go here
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
#from us import states
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re
import networkx as nx

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import minimum_spanning_tree
from itertools import combinations

In [ ]:
clustertype="ClusterwoSCC"
cluster_filename = "PM25-Speciated/reconcensusclustering/"+clustertype+".pamcon"

In [ ]:
def create_node_cluster_dict(cluster_filename):
    node_cluster = {}
    
    with open(cluster_filename, 'r') as file:
        for cluster_num, line in enumerate(file, start=1):
            # Split the line into individual node numbers
            nodes = line.strip().split()
            
            for node in nodes:
                # Convert node to integer if needed (remove int() if nodes are strings)
                node_cluster[int(node)] = cluster_num
                
    return node_cluster

# Example usage:
node_cluster_dict = create_node_cluster_dict(cluster_filename)
print(node_cluster_dict)

In [ ]:
df = pivot_df
df['Cluster'] = df['Unnamed: 0'].map(node_cluster_dict)
df.head()

In [ ]:
# 2. Build the graph
G = nx.Graph()
for _, row in df.iterrows():
#     G.add_node(row['Unnamed: 0'], cluster=row['Cluster'])
    G.add_node(row['Unnamed: 0'], cluster=row['Cluster'], name=row['PROFILE_NAME'])

for cluster in df['Cluster'].unique():
    cluster_nodes = df[df['Cluster'] == cluster]['Unnamed: 0'].values
    cluster_data = df[df['Cluster'] == cluster].select_dtypes(include=[np.number]).values
    
    if len(cluster_nodes) > 1:
        n_neighbors = min(2, len(cluster_nodes) - 1)
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(cluster_data)
        distances, indices = nbrs.kneighbors(cluster_data)
        
        for i in range(len(cluster_nodes)):
            for j_idx in range(n_neighbors):
                j = indices[i][j_idx]
                if i != j:
                    G.add_edge(cluster_nodes[i], cluster_nodes[j], weight=1.0/(distances[i][j_idx] + 1e-6))

# 3. Ensure full connectivity within clusters (optional)
for cluster in df['Cluster'].unique():
    cluster_nodes = list(df[df['Cluster'] == cluster]['Unnamed: 0'].values)
    if len(cluster_nodes) > 1 and not nx.is_connected(G.subgraph(cluster_nodes)):
        for i in range(len(cluster_nodes)):
            for j in range(i + 1, len(cluster_nodes)):
                if not G.has_edge(cluster_nodes[i], cluster_nodes[j]):
                    G.add_edge(cluster_nodes[i], cluster_nodes[j], weight=1.0)

In [ ]:
y = [i+'-'+str(j) for i,j in zip(df['PROFILE_NAME'],df['PROFILE_CODE'])]
# y = [str(i) for i in df['PROFILE_CODE']]
labels = dict(zip(range(len(y)), y))

nx.set_node_attributes(G, labels, 'labels')

# z = [str(i) for i in df['Cluster']]
# cluster = dict(zip(range(len(z)), z))
# nx.set_node_attributes(G, cluster, 'Cluster')

filename = "PM25-Speciated/"+"new_concensus-"+clustertype+'.gexf'
print("Writing gephi...."+filename)
nx.write_gexf(G, filename)
print("Done....")

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# view cluster (without SCC)

In [ ]:
# Get unique cluster numbers
unique_clusters = df['Cluster'].unique()

for cluster_num in sorted(unique_clusters):
    # Get rows belonging to this cluster
    cluster_rows = df[df['Cluster'] == cluster_num]
    
    # Skip if cluster is empty (or handle NaN if present)
    if len(cluster_rows) == 0:
        continue
    
    fig, ax = plt.subplots(figsize=(20, 20))
    
    # Select columns for boxplot (adjust 2:45 as needed)
    boxplot_data = cluster_rows.iloc[:, 6:]
    boxplot_data.boxplot(ax=ax, grid=False)
    ax.grid(False)
    
    plt.title(f"Boxplot for Cluster {cluster_num}")
    plt.xticks(rotation=90)
    
    # Create table area
    divider = make_axes_locatable(ax)
    ax_table = divider.append_axes("right", size="40%", pad=0.0)
    ax_table.axis('off')
    
    # Create table - adjust 'PROFILE_NAME' to your actual column name
    table_data = cluster_rows[['PROFILE_NAME']]
    table = ax_table.table(
        cellText=table_data.values,
        colLabels=table_data.columns,
        cellLoc='left',
        loc='right'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(12)
#     table.scale(2, 2)
    
#     plt.tight_layout()
    plt.show()

In [ ]:
df.columns

In [ ]:
# Define metal and non-metal elements
metal_elements = [
    'Aluminum', 'Antimony', 'Arsenic', 'Barium', 'Cadmium', 'Calcium',
    'Chromium', 'Cobalt', 'Copper', 'Indium', 'Iron', 'Lead', 'Magnesium',
    'Manganese', 'Mercury', 'Molybdenum', 'Nickel', 'Silver', 'Sodium',
    'Strontium', 'Tin', 'Titanium', 'Vanadium', 'Zinc', 'Zirconium'
]

non_metal_elements = [
    'Ammonium', 'Chloride ion', 'Elemental Carbon', 'Nitrate', 'Organic carbon',
    'Phosphorus', 'Potassium', 'Potassium ion', 'Selenium', 'Silicon',
    'Sodium ion', 'Sulfate', 'Sulfur'
]

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_boxplot(df, cluster_num, element_list, title_suffix):
    """Plots a boxplot for a given cluster and element list."""
    cluster_rows = df[df['Cluster'] == cluster_num]
    
    # Skip if cluster is empty
    if len(cluster_rows) == 0:
        return
    
    # Select only the relevant columns
    boxplot_data = cluster_rows[element_list]
    
    fig, ax = plt.subplots(figsize=(15, 8))
    boxplot_data.boxplot(ax=ax, grid=False)
    ax.grid(False)
    
    plt.title(f"Boxplot for Cluster {cluster_num} ({title_suffix})")
    plt.xticks(rotation=90)
    
    # Create table area
    divider = make_axes_locatable(ax)
    ax_table = divider.append_axes("right", size="40%", pad=0.0)
    ax_table.axis('off')
    
    # Display profile names for reference
    table_data = cluster_rows[['PROFILE_NAME']]
    table = ax_table.table(
        cellText=table_data.values,
        colLabels=table_data.columns,
        cellLoc='left',
        loc='right'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    
    plt.show()

# Get unique cluster numbers
unique_clusters = df['Cluster'].unique()

## Metal

In [ ]:
for cluster_num in sorted(unique_clusters):
    plot_boxplot(df, cluster_num, metal_elements, "Metals")
#     plot_boxplot(df, cluster_num, non_metal_elements, "Non-Metals")


## Non Metal

In [ ]:
for cluster_num in sorted(unique_clusters):
#     plot_boxplot(df, cluster_num, metal_elements, "Metals")
    plot_boxplot(df, cluster_num, non_metal_elements, "Non-Metals")


# Top K important species

In [ ]:
# Number of top important species to display
k_species = 5

# Get the unique cluster numbers
unique_clusters = df['Cluster'].unique()

for cluster_num in sorted(unique_clusters):
    # Get rows belonging to this cluster
    cluster_rows = df[df['Cluster'] == cluster_num]

    # Skip if no rows found (or handle NaN clusters)
    if cluster_rows.empty:
        continue

    # Select species-related columns; adjust the column indices as needed.
    # Here, we assume the species data is stored in all columns starting from index 6.
    species_data = cluster_rows.iloc[:, 6:]

    # Compute SVD on the species data matrix (ensure the data is numeric)
    U, S, Vt = np.linalg.svd(species_data, full_matrices=False)

    # Calculate column importance using the SVD components.
    # We square Vt, weight by the singular values and sum along the rows to aggregate importance.
    column_importance = np.sum(np.matmul(np.diag(S), Vt**2), axis=0)

    # Identify the indices of the top k important species
    important_columns_indices = np.argsort(-column_importance)[:k_species]

    # Get the corresponding column names
    important_columns = species_data.columns[important_columns_indices]

    print(f"Important species for Cluster {cluster_num}: {list(important_columns)}")

    # Create a figure and axes for the boxplot
    fig, ax = plt.subplots(figsize=(20, 10))

    # Create the boxplot using only the key species
    ax.boxplot(cluster_rows[important_columns])
    ax.grid(False)
    ax.set_xticklabels(important_columns, rotation=90)
    plt.title(f"Boxplot for key species in Cluster {cluster_num}")

    # Create a divider for the existing axes instance to add a table on the right
    divider = make_axes_locatable(ax)
    ax_table = divider.append_axes("right", size="20%", pad=0.0)
    ax_table.axis("off")

    # Create the table using the PROFILE_NAME column (adjust the column name if needed)
    table_data = cluster_rows[['PROFILE_NAME']]
    table = ax_table.table(
        cellText=table_data.values,
        colLabels=table_data.columns,
        cellLoc='left',
        loc='right'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(12)
#     table.scale(2, 2)

    plt.tight_layout()
    plt.show()
